In [1]:
parentPath = "/Users/Aran/Documents/workspace/MLExpDir/machine-learning-with-spark"

In [2]:
rawData = sc.textFile("%s/ml-100k/u.data" % parentPath)

In [3]:
rawData.first()

u'196\t242\t3\t881250949'

In [4]:
rawRatings = rawData.map(lambda x: x.split("\t")[:3]) 

In [5]:
rawRatings.first()

[u'196', u'242', u'3']

In [6]:
from pyspark.mllib.recommendation import ALS, Rating

In [7]:
ratings = rawRatings.map(lambda x: Rating(int(x[0]), int(x[1]), float(x[2])))

In [8]:
ratings.first()

Rating(user=196, product=242, rating=3.0)

In [9]:
rank = 50  # The rank of factor matrix
iterations = 10
lamda = 0.01
model = ALS.train(ratings, rank, iterations, lamda)

In [10]:
userFeatures = model.userFeatures()

In [11]:
userFeatures.count()

943

In [12]:
model.predict(789, 123)

2.4851097695635453

In [13]:
ratings

PythonRDD[6] at RDD at PythonRDD.scala:43

In [14]:
from pyspark.rdd import RDD

In [15]:
?RDD.count

In [16]:
userId = 789
K = 10
topKRecs = model.recommendProducts(userId, K)

In [17]:
topKRecs

[Rating(user=789, product=192, rating=5.914977114560879),
 Rating(user=789, product=429, rating=5.426148504193856),
 Rating(user=789, product=187, rating=5.4138449324295985),
 Rating(user=789, product=663, rating=5.343478689987497),
 Rating(user=789, product=675, rating=5.302221323375578),
 Rating(user=789, product=32, rating=5.192621350002867),
 Rating(user=789, product=108, rating=5.14498958808262),
 Rating(user=789, product=661, rating=5.133586034482052),
 Rating(user=789, product=511, rating=5.132557695056045),
 Rating(user=789, product=521, rating=5.1323243269839285)]

In [18]:
movies = sc.textFile("%s/ml-100k/u.item" % parentPath)

In [19]:
titles = movies.map(lambda line: line.split("|")[:2]).map(lambda array: (int(array[0]), array[1])).collectAsMap()

In [20]:
titles[123]

u'Frighteners, The (1996)'

In [21]:
moviesForUser = ratings.keyBy(lambda rating: rating.user).lookup(789)

In [22]:
len(moviesForUser)

33

In [23]:
moviesForUser[0]

Rating(user=789, product=1012, rating=4.0)

In [24]:
topKRateTemp = sorted(moviesForUser, key=lambda rating: rating.rating, reverse=True)[:10]
topKRating = [(titles[rating.product], rating.rating) for rating in topKRateTemp]
for item in topKRating:
    print item 

(u'Godfather, The (1972)', 5.0)
(u'Trainspotting (1996)', 5.0)
(u'Dead Man Walking (1995)', 5.0)
(u'Star Wars (1977)', 5.0)
(u'Swingers (1996)', 5.0)
(u'Leaving Las Vegas (1995)', 5.0)
(u'Bound (1996)', 5.0)
(u'Fargo (1996)', 5.0)
(u'Last Supper, The (1995)', 5.0)
(u'Private Parts (1997)', 4.0)


In [25]:
topKRecsTemp = [(titles[rating.product], rating.rating) for rating in topKRecs]
for item in topKRecsTemp:
    print item

(u'Raging Bull (1980)', 5.914977114560879)
(u'Day the Earth Stood Still, The (1951)', 5.426148504193856)
(u'Godfather: Part II, The (1974)', 5.4138449324295985)
(u'Being There (1979)', 5.343478689987497)
(u'Nosferatu (Nosferatu, eine Symphonie des Grauens) (1922)', 5.302221323375578)
(u'Crumb (1994)', 5.192621350002867)
(u'Kids in the Hall: Brain Candy (1996)', 5.14498958808262)
(u'High Noon (1952)', 5.133586034482052)
(u'Lawrence of Arabia (1962)', 5.132557695056045)
(u'Deer Hunter, The (1978)', 5.1323243269839285)


In [26]:
import numpy as np
a = np.array([[1, 2, 3]])
b = np.array([[1], [2], [3]])

In [27]:
b.shape

(3, 1)

In [28]:
b.dot(a)

array([[1, 2, 3],
       [2, 4, 6],
       [3, 6, 9]])

In [29]:
c = np.array([3,4,12])
print np.linalg.norm(c)
d = np.array([1, 2, 3])
print np.dot(c, d)

13.0
47


In [30]:
# Assume the parameters are all 1-d ndarray
def cosineSimilarity(vec1, vec2):
    return np.dot(vec1, vec2)/(np.linalg.norm(vec1)*np.linalg.norm(vec2)) 

In [31]:
cosineSimilarity(c, c)

1.0

In [32]:
?RDD.lookup

In [33]:
rdd = sc.parallelize([(1,2), (1,3)],2)
rdd.lookup(1)

[2, 3]

In [34]:
itemId = 567
# The lookup result only has one item, which is the item factor
productFeatures = model.productFeatures()
itemVector = productFeatures.lookup(itemId)[0]

In [35]:
itemVector

array('d', [0.891995906829834, -0.5106445550918579, 0.13697193562984467, -0.21415044367313385, 0.402266263961792, -0.08696794509887695, 0.2693471610546112, -0.5349130034446716, 1.7306143045425415, -0.5012935996055603, -0.3326531946659088, 0.38982290029525757, -0.5004117488861084, -1.4292975664138794, 0.24958425760269165, 0.6228033304214478, -0.43864649534225464, 0.4975425601005554, 0.5161775350570679, 0.5634575486183167, 0.10650111734867096, 0.157758429646492, 0.012060972861945629, 0.5797746181488037, 0.28802481293678284, -0.505713164806366, -0.13843928277492523, 0.06099162995815277, 0.01486317440867424, -0.3673551082611084, 0.2579803764820099, -0.5968518257141113, 0.3906335234642029, 0.35944733023643494, -0.05501624941825867, -0.32503125071525574, -0.5100358724594116, -0.0689019113779068, -0.008764437399804592, -0.32559749484062195, 0.9317123293876648, 0.6578829288482666, -0.17786917090415955, -0.6433614492416382, 1.3173636198043823, 1.1199944019317627, -0.06760596483945847, 0.3162276

In [36]:
cosineSimilarity(itemVector, itemVector)

0.99999999999999978

In [37]:
sims = productFeatures.map(lambda itemFeature: (itemFeature[0], cosineSimilarity(itemFeature[1], itemVector)))

In [38]:
sims.first()

(8, 0.56101400562089143)

In [39]:
sortedSims = sims.takeOrdered(K, key=lambda x: -x[1])

In [40]:
sortedSims

[(567, 0.99999999999999978),
 (349, 0.68876136028135715),
 (563, 0.68292533107866948),
 (183, 0.68021911057140749),
 (433, 0.68017110002588799),
 (323, 0.6737082347194826),
 (201, 0.67336613176037441),
 (1083, 0.67145430130942085),
 (219, 0.66558034918890285),
 (195, 0.66555767436390623)]

In [41]:
titles[itemId]

u"Wes Craven's New Nightmare (1994)"

In [42]:
sortedSims2 = sims.takeOrdered(K+1, key=lambda x: -x[1])
[(titles[item[0]], item[1]) for item in sortedSims2[1:]]

[(u'Hard Rain (1998)', 0.68876136028135715),
 (u"Stephen King's The Langoliers (1995)", 0.68292533107866948),
 (u'Alien (1979)', 0.68021911057140749),
 (u'Heathers (1989)', 0.68017110002588799),
 (u"Dante's Peak (1997)", 0.6737082347194826),
 (u'Evil Dead II (1987)', 0.67336613176037441),
 (u'Albino Alligator (1996)', 0.67145430130942085),
 (u'Nightmare on Elm Street, A (1984)', 0.66558034918890296),
 (u'Terminator, The (1984)', 0.66555767436390623),
 (u'Fierce Creatures (1997)', 0.65982079382944425)]

In [43]:
actualRating = moviesForUser[0]

In [44]:
predictedRating = model.predict(789, actualRating.product)

In [45]:
predictedRating

4.033903800290711

In [46]:
squaredError = np.square(predictedRating - actualRating.rating)

In [47]:
squaredError

0.0011494676741523886

In [48]:
ratings.first()

Rating(user=196, product=242, rating=3.0)

In [49]:
usersProducts = ratings.map(lambda rating: (rating.user, rating.product))
predictions = model.predictAll(usersProducts).map(lambda rating: ((rating.user, rating.product), rating.rating))

In [50]:
ratingsAndPredictions = ratings.map(lambda rating: ((rating.user, rating.product), rating.rating)).join(predictions)

In [51]:
ratingsAndPredictions.first()

((360, 14), (5.0, 4.8398683566215315))

In [52]:
MSE = ratingsAndPredictions.map(lambda x: np.square(x[1][0] - x[1][1])).reduce(lambda x, y: x+y)/ratingsAndPredictions.count()

In [53]:
np.sqrt(MSE)

0.29066364460079852

In [54]:
abc = [1,2,3]
for i, a in enumerate(abc):
    print i,a

0 1
1 2
2 3


In [55]:
def avgPrecisionK(actual, predicted, k):
    predK = predicted[:k]
    score = 0.0
    numHits = 0.0
    for i, p in enumerate(predK):
        if p in actual:
            numHits += 1.0
            score += numHits / (i + 1.0)
    if len(actual) == 0:
        return 1.0
    else:
        return score/min(len(actual), k)

In [56]:
actualMovies = [rating.product for rating in moviesForUser]
predictedMovies = [rating.product for rating in topKRecs]

In [57]:
avgPrecisionK(actualMovies, predictedMovies, 10)

0.0

In [58]:
itemFactors = model.productFeatures().map(lambda idAndFactor: idAndFactor[1]).collect()
itemMatrix = np.array(itemFactors)
itemMatrix.shape

(1682, 50)

In [59]:
imBroadcast = sc.broadcast(itemMatrix)

In [60]:
def getRecommendIds(userIdAndFactor):
    userVector = np.array(userIdAndFactor[1])
    scores = np.dot(imBroadcast.value, userVector)
    sortedWithId = sorted([(score, i) for i, score in enumerate(scores)], key=lambda x: -x[0])
    recommendedIds = [(index+1) for score, index in sortedWithId]
    return (userIdAndFactor[0], recommendedIds)

In [61]:
allRecs = model.userFeatures().map(lambda userIdAndFactor: getRecommendIds(userIdAndFactor))

In [62]:
userMovies = ratings.map(lambda rating: (rating.user, rating.product)).groupBy(lambda x: x[0])

In [63]:
def getMAPK(k):
    return allRecs.join(userMovies).map(lambda x: avgPrecisionK([y[1] for y in x[1][1]], x[1][0], k)).reduce(lambda x, y: x+y)/allRecs.count()

In [64]:
getMAPK(K)

0.030451404669326186

In [65]:
from pyspark.mllib.evaluation import RegressionMetrics

In [66]:
predictedAndTrue = ratingsAndPredictions.map(lambda x: x[1])
regressionMetrics = RegressionMetrics(predictedAndTrue)

In [67]:
print "Mean Squared Error = %f" % regressionMetrics.meanSquaredError
print "Root Mean Squared Error = %f" % regressionMetrics.rootMeanSquaredError

Mean Squared Error = 0.084485
Root Mean Squared Error = 0.290664


In [68]:
from pyspark.mllib.evaluation import RankingMetrics

In [69]:
list(userMovies.first()[1])[:1]

[(2, 292)]

In [70]:
predictedAndTrueForRanking = allRecs.join(userMovies).map(lambda x: (x[1][0], [y[1] for y in x[1][1]]))

In [71]:
rankingMetrics = RankingMetrics(predictedAndTrueForRanking)
print "Mean Average Precision = %f" % rankingMetrics.meanAveragePrecision

Mean Average Precision = 0.071669


In [72]:
getMAPK(2000)

0.0716694171640423